In [1]:
pip install keras 

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install tensorflow

In [3]:
#importing required libraries and modules
import numpy as np
import keras
import keras.backend as k
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
from keras.optimizers import Adam
from keras.preprocessing import image
import cv2
import datetime

In [4]:
# Building a simple Convolutional Neural Network (CNN) using the Keras sequential API

model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [5]:
#setting up an image data generator using Keras's ImageDataGenerator for training and testing

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        "C:/Users/user/Desktop/Face-mask Detection/Dataset/train",  #path of train set
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        "C:/Users/user/Desktop/Face-mask Detection/Dataset/test",  #path of test set
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')

model_saved=model.fit_generator(
        training_set,
        epochs=10,
        validation_data=test_set,

        )

Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


C:\Users\user\AppData\Local\Temp\ipykernel_19264\1744862153.py:24: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_saved=model.fit_generator(


Epoch 1/10


83/83 [==============================] - 32s 354ms/step - loss: 0.4057 - accuracy: 0.7892 - val_loss: 0.1742 - val_accuracy: 0.9330
Epoch 2/10
83/83 [==============================] - 25s 304ms/step - loss: 0.1300 - accuracy: 0.9597 - val_loss: 0.1045 - val_accuracy: 0.9639
Epoch 3/10
83/83 [==============================] - 25s 305ms/step - loss: 0.0999 - accuracy: 0.9665 - val_loss: 0.0857 - val_accuracy: 0.9639
Epoch 4/10
83/83 [==============================] - 26s 311ms/step - loss: 0.0898 - accuracy: 0.9696 - val_loss: 0.0977 - val_accuracy: 0.9639
Epoch 5/10
83/83 [==============================] - 25s 306ms/step - loss: 0.0781 - accuracy: 0.9711 - val_loss: 0.1373 - val_accuracy: 0.9485
Epoch 6/10
83/83 [==============================] - 25s 306ms/step - loss: 0.0787 - accuracy: 0.9741 - val_loss: 0.0881 - val_accuracy: 0.9639
Epoch 7/10
83/83 [==============================] - 26s 314ms/step - loss: 0.0673 - accuracy: 0.9734 - val_loss: 0.1310 - val_accuracy: 0.95

### built-in model

In [6]:
#Save the model
model.save('mymodel.h5',model_saved) 

#Load the model
mymodel=load_model('mymodel.h5')

C:\Users\user\anaconda3\New folder\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### model accuracy

In [7]:
# Print training and validation accuracy

training_accuracy = model_saved.history['accuracy']
validation_accuracy = model_saved.history['val_accuracy']

print("Training Accuracy:", training_accuracy[-1]*100)
print("Validation Accuracy:", validation_accuracy[-1]*100)

Training Accuracy: 96.49924039840698
Validation Accuracy: 94.84536051750183


In [8]:
#loading two test images, converting them to NumPy arrays, and then using the model to make predictions

test_image=image.load_img(r"C:/Users/user/Desktop/Face-mask Detection/Dataset/test/without_mask/31.jpg",  # path of image from the "without_mask" class
                          target_size=(150,150,3))  
test_image=image.load_img(r"C:/Users/user/Desktop/Face-mask Detection/Dataset/test/with_mask/456-with-mask.jpg",  #path of image from the "with_mask" class
                          target_size=(150,150,3))   
test_image
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
mymodel.predict(test_image)[0][0]

1/1 [==============================] - 0s 238ms/step


0.0

In [9]:
# Implementing live detection of face mask

mymodel=load_model('mymodel.h5') # Loading the pre-trained model
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml') #Loading the Haar Cascade classifier for face detection.
while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    if cv2.waitKey(1)==ord('q'):   # press 'q' key to terminate the loop   
        break   
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 29ms/step
